# **Assignment: SemEval 2025 Task 9 - Food Hazard Detection Challenge**

This assignment tasks us with participating in the **SemEval 2025 Task 9: The Food Hazard Detection Challenge**. The goal is to explore and classify food hazard incidents based on text data, specifically focusing on food recall reports from various web sources. we will work with both short texts (titles) and long texts (full descriptions) provided in the dataset, performing a series of benchmark analyses using basic and advanced machine learning algorithms.

The assignment is structured into two key benchmark analyses:
1. **Short Texts (Titles)**: We will evaluate the performance of text classification models on food hazard prediction based solely on the titles of food incident reports.
2. **Long Texts (Full Descriptions)**: We will extend the analysis to the full recall descriptions to assess whether longer text data improves the classification performance.

We will train and evaluate models such as **Logistic Regression** and **Naive Bayes** for basic machine learning algorithms, and an **advanced BERT-based model** for more sophisticated, transformer-based classification. The performance of the models will be evaluated based on the **macro F1-score** for both hazard and product category prediction.

By the end of the assignment, we will compare the results from these models, determine which approach performs best, and prepare our predictions for submission. The final submission will include the prediction results, along with an explanation of the models used, evaluation metrics, and our ranking on the competition leaderboard.
